In [49]:
import geopandas as gpd
import ee
from shapely.geometry import Polygon,MultiPolygon
import pprint
import matplotlib.pyplot as plt
import IPython.display as disp
from shapely.ops import unary_union
import shapely
import json


GEE_DEFAULT_CRS = "EPSG:4326"

In [41]:
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [42]:
def get_dummy_collection():
    geoJSON = {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [
                  -52.4646,
                  -3.6592
                ],
                [
                  -51.7779,
                  -3.6592
                ],
                [
                  -51.7779,
                  -3.2863
                ],
                [
                  -52.4646,
                  -3.2863
                ],
                [
                  -52.4646,
                  -3.6592
                ]
              ]
            ]
          }
        }
      ]
    }

    coords = geoJSON['features'][0]['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coords)
    ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                      .filterBounds(aoi)
                      .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                      .first()
                      .clip(aoi))
    ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                      .filterBounds(aoi)
                      .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                      .first()
                      .clip(aoi))

    ffa_db.bandNames().getInfo()

    coords = ffa_db.getInfo()['properties']['system:footprint']['coordinates'][0]
    footprint = Polygon(coords)
    print(footprint)
    return ffa_db,aoi

In [43]:
# polygon is too annoying for argparse: easier to create tiles if this is a rectangle
def get_footprint_dataset(poly):
    footprint_polygon = gpd.GeoSeries([poly])
    footprint = gpd.GeoDataFrame({'geometry': footprint_polygon})
    return footprint


def intersect_polygons_with_footprint(footprint, polygons):
    # need to adjust crs for both polygons:
    polygons_image = footprint.overlay(polygons, how='intersection')
    return polygons_image




In [177]:
def mask_collection(image, polygons,aoi):
    #get footprint from image:
    coords = image.getInfo()['properties']['system:footprint']['coordinates'][0]
    footprint_poly = Polygon(coords)
    footprint = get_footprint_dataset(footprint_poly)
    footprint = footprint.set_crs(GEE_DEFAULT_CRS,allow_override=True)
    footprint = footprint.to_crs(GEE_DEFAULT_CRS)
    polygons = polygons.set_crs(GEE_DEFAULT_CRS,allow_override=True)
    polygons = polygons.to_crs(GEE_DEFAULT_CRS)

    # intersect the footprint with all polygons from polygons:
    polygons_image = intersect_polygons_with_footprint(footprint, polygons)

    print(polygons_image['geometry'][:])
    
    geomlist = list(polygons_image['geometry'][:])
    multipoly = unary_union(geomlist)
    
    # convert to ee multipolygon 
    geojson = json.dumps(shapely.geometry.mapping(multipoly))
    geojson = json.loads(geojson)
    multipoly = ee.Geometry.MultiPolygon(geojson['coordinates'])
    
    feature = ee.FeatureCollection([ee.Feature(multipoly, {'deforestation':1})])
    #print(feature.propertyNames())
    
    #feature to image: 
    deforestimg = feature.reduceToImage(**{'properties': ['deforestation'],
        'reducer': ee.Reducer.first()})
    
    print(deforestimg.getInfo())
    
    geojson = json.dumps(shapely.geometry.mapping(footprint_poly))
    geojson = json.loads(geojson)
    footprint_poly = ee.Geometry.Polygon(geojson['coordinates'])
    
    # need to be clipped to area of interest: 
    # two nots as first one does conversion to black second should the trick
    deforestimg = deforestimg.clip(footprint_poly).Not().Not()
    
    location = aoi.centroid().coordinates().getInfo()[::-1]

    # Make an RGB color composite image (VV,VH,VV/VH).
    rgb = ee.Image.rgb(image.select('VV'),
                       image.select('VH'),
                       image.select('VV').divide(image.select('VH')))

    # Create the map object.
    m = folium.Map(location=location, zoom_start=12)

    # Add the S1 rgb composite to the map object.
    m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

    # Add a layer control panel to the map.
    #m.add_child(folium.LayerControl())
    
    #m.add_ee_layer(multipoly, {}, 'mask')
    m.add_ee_layer(deforestimg, {}, 'mask')

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)
    
    segmented_image = image.addBands(deforestimg).clip(footprint_poly)
    
    print(segmented_image.getInfo())
    

    return segmented_image

In [161]:
poly = '/Users/fredericboesel/Documents/Data Science Master/Herbstsemester 2021/AI4Good/ai4good/data/alertas_2020_com_municipio_e_coordenadas/alertas_2020_com_municipio_e_coordenadas.shp'
polygons = gpd.read_file(poly)
print(polygons)

       IDAlerta     Bioma    Estado                 Municipio  AreaHa  \
0        118901  AMAZÔNIA      PARÁ         NOVO REPARTIMENTO  851.79   
1        118902  AMAZÔNIA      PARÁ    BOM JESUS DO TOCANTINS   60.47   
2        118910  AMAZÔNIA  AMAZONAS  SÃO GABRIEL DA CACHOEIRA    0.81   
3        118912  AMAZÔNIA  AMAZONAS  SÃO GABRIEL DA CACHOEIRA    1.57   
4        118913  AMAZÔNIA  AMAZONAS  SÃO GABRIEL DA CACHOEIRA    0.58   
...         ...       ...       ...                       ...     ...   
74246    305112  AMAZÔNIA      PARÁ                     ANAPU    1.33   
74247    305119  AMAZÔNIA      PARÁ              MEDICILÂNDIA    0.60   
74248    305122  AMAZÔNIA      PARÁ              MEDICILÂNDIA    2.76   
74249    305125  AMAZÔNIA      PARÁ               BRASIL NOVO   11.60   
74250    305144  AMAZÔNIA      PARÁ              PORTO DE MOZ    4.16   

       AnoDetec   DataDetec          x         y  \
0        2020.0  2020-03-01 -50.983725 -4.334486   
1        2020.0  20

In [181]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [182]:
im,aoi = get_dummy_collection()

POLYGON ((-52.4646 -3.6592, -51.7779 -3.659199999999999, -51.7779 -3.2863, -52.46459999999999 -3.286300000000001, -52.4646 -3.6592))


In [183]:
im = mask_collection(im,polygons,aoi)

0      POLYGON ((-52.08959 -3.44829, -52.08970 -3.447...
1      MULTIPOLYGON (((-52.08753 -3.29840, -52.08778 ...
2      MULTIPOLYGON (((-52.20945 -3.64243, -52.20942 ...
3      POLYGON ((-52.08464 -3.58306, -52.08484 -3.583...
4      MULTIPOLYGON (((-51.83585 -3.49727, -51.83636 ...
                             ...                        
262    POLYGON ((-52.05925 -3.45422, -52.05919 -3.453...
263    POLYGON ((-52.27317 -3.37336, -52.27301 -3.373...
264    POLYGON ((-52.13391 -3.58405, -52.13416 -3.584...
265    MULTIPOLYGON (((-52.03763 -3.65611, -52.03763 ...
266    MULTIPOLYGON (((-52.24542 -3.57919, -52.24536 ...
Name: geometry, Length: 267, dtype: geometry
{'type': 'Image', 'bands': [{'id': 'first', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7635, 4133], 'origin': [3703, 5510], 'crs': 'EPSG:32722', 'crs_transform': [10, 0, 300241.5790180225, 0, -10, 9691832.441629494]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7635, 4133], 'origin': [3703, 5510], 'crs': 'EPSG:32722', 'crs_transform': [10, 0, 300241.5790180225, 0, -10, 9691832.441629494]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8, 4], 'origin': [12, 1], 'crs': 'EPSG:32722', 'crs_transform': [-12631.59647357976, -4323.139024902601, 589236.8633570769, 2751.200076887384, -20052.12939104438, 9636717.95695303]}, {'id': 'first', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [2, 1], 'origin': [-53, -4], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200803T090644_20200803T090